In [2]:
# Importing Libraries


import datetime
import sqlite3
from tkcalendar import DateEntry
from tkinter import *
from tkinter import ttk
import tkinter.messagebox as mb

In [3]:
# -------------------- Database Setup -------------------- #

# Connecting to the Database
connector = sqlite3.connect("PersonalExpenseTracker.db")
cursor = connector.cursor()

# Creating Expense Tracker Table without 'Payee' and renaming 'Description' to 'ItemBought'
cursor.execute(
    '''
    CREATE TABLE IF NOT EXISTS ExpenseTracker (
        ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        Date DATE,
        ItemBought TEXT,
        Amount FLOAT,
        ModeOfPayment TEXT
    )
    '''
)

# Creating Bank Balance Table (Assuming only one balance record)
cursor.execute(
    '''
    CREATE TABLE IF NOT EXISTS BankBalance (
        ID INTEGER PRIMARY KEY CHECK (ID = 1),
        Balance FLOAT
    )
    '''
)

# Initialize Bank Balance if not present
cursor.execute("SELECT * FROM BankBalance WHERE ID = 1")
if not cursor.fetchone():
    cursor.execute("INSERT INTO BankBalance (ID, Balance) VALUES (1, 0.0)")
    connector.commit()

# Creating To-Buy List Table
cursor.execute(
    '''
    CREATE TABLE IF NOT EXISTS ToBuyList (
        ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        ItemName TEXT
    )
    '''
)
connector.commit()

In [4]:
# -------------------- GUI Setup -------------------- #

# Initialize the GUI window
root = Tk()
root.title('Personal Expense Tracker')
root.geometry('1470x700')  # Increased window size
root.configure(bg='#ECF0F1')  # Soft White background
root.resizable(0, 0)

''

In [5]:
# -------------------- Styling -------------------- #

style = ttk.Style()
style.theme_use('clam')  # Use 'clam' for better theming support

# Define color palette
PRIMARY_COLOR = '#2C3E50'       # Navy Blue
SECONDARY_COLOR = '#ECF0F1'     # Soft White
ACCENT_COLOR = '#27AE60'        # Emerald Green
HIGHLIGHT_COLOR = '#E67E22'     # Orange
TEXT_COLOR = '#34495E'           # Charcoal
BUTTON_HOVER_COLOR = '#1A252F'   # Darker Navy Blue

# Configure styles
style.configure('TFrame', background=SECONDARY_COLOR)
style.configure('TLabel', background=SECONDARY_COLOR, foreground=TEXT_COLOR, font=('Helvetica', 12))
style.configure('Header.TLabel', background=SECONDARY_COLOR, foreground=PRIMARY_COLOR, font=('Helvetica', 20, 'bold'))
style.configure('TButton',
                foreground='white',
                background=PRIMARY_COLOR,
                font=('Helvetica', 12, 'bold'),
                padding=6)
style.map('TButton',
          background=[('active', BUTTON_HOVER_COLOR)],
          foreground=[('active', 'white')])

style.configure('TEntry', 
                foreground=TEXT_COLOR, 
                font=('Helvetica', 12))
style.configure('TCombobox',
                foreground=TEXT_COLOR,
                font=('Helvetica', 12),
                fieldbackground='white')

In [6]:
# -------------------- Functions -------------------- #

def get_current_balance():
    cursor.execute("SELECT Balance FROM BankBalance WHERE ID = 1")
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        return 0.0

def set_balance(new_balance):
    cursor.execute("UPDATE BankBalance SET Balance = ? WHERE ID = 1", (new_balance,))
    connector.commit()
    balance_label.config(text=f"Current Balance: ₹{get_current_balance():.2f}")

def list_all_expenses():
    for item in table.get_children():
        table.delete(item)
    cursor.execute('SELECT * FROM ExpenseTracker ORDER BY Date DESC')
    data = cursor.fetchall()
    for values in data:
        table.insert('', END, values=values)

def list_to_buy_items():
    for item in to_buy_table.get_children():
        to_buy_table.delete(item)
    cursor.execute('SELECT * FROM ToBuyList')
    data = cursor.fetchall()
    for item in data:
        to_buy_table.insert('', END, values=item)

def add_another_expense():
    if not date.get() or not item_bought.get() or not amnt.get() or not MoP.get():
        mb.showerror('Fields Empty', "Please fill all the fields before adding an expense.")
    else:
        try:
            amount = float(amnt.get())
            if amount <= 0:
                mb.showerror('Invalid Amount', "Amount must be greater than zero.")
                return
        except ValueError:
            mb.showerror('Invalid Input', "Please enter a valid amount.")
            return
        current_balance = get_current_balance()
        if amount > current_balance:
            mb.showerror('Insufficient Balance', "You do not have enough balance for this expense.")
            return
        try:
            # Insert expense into the database
            cursor.execute(
                'INSERT INTO ExpenseTracker (Date, ItemBought, Amount, ModeOfPayment) VALUES (?, ?, ?, ?)',
                (date.get_date(), item_bought.get(), amount, MoP.get())
            )
            connector.commit()
            # Deduct from balance
            set_balance(current_balance - amount)
            # Refresh expense list
            list_all_expenses()
            # Clear fields
            clear_fields()
            mb.showinfo('Expense Added', 'Expense has been added and balance updated.')
            # Debug: Confirm addition
            print(f"Added Expense: Date={date.get_date()}, ItemBought={item_bought.get()}, Amount={amount}, ModeOfPayment={MoP.get()}")
        except sqlite3.Error as e:
            mb.showerror('Database Error', f"An error occurred while adding the expense: {e}")
            print(f"SQLite Error: {e}")

def edit_expense():
    if not table.selection():
        mb.showerror('No Selection', 'Please select an expense to edit.')
        return

    selected = table.selection()
    if not selected:
        mb.showerror('No Selection', 'Please select an expense to edit.')
        return

    current_selected = table.item(selected[0])
    values = current_selected['values']

    # Debug: Print selected values
    print(f"Selected Expense for Editing: ID={values[0]}, Date={values[1]}, ItemBought={values[2]}, Amount={values[3]}, ModeOfPayment={values[4]}")

    # Open a new window for editing
    edit_window = Toplevel(root)
    edit_window.title("Edit Expense")
    edit_window.geometry('400x400')
    edit_window.configure(bg=SECONDARY_COLOR)
    edit_window.resizable(0, 0)

    # Labels and Entries
    Label(edit_window, text='Date:', font=('Helvetica', 12), bg=SECONDARY_COLOR).pack(pady=10)
    try:
        parsed_date = datetime.datetime.strptime(values[1], '%Y-%m-%d').date()
    except ValueError:
        parsed_date = datetime.datetime.now().date()
    edit_date = DateEntry(edit_window, date=parsed_date, font=('Helvetica', 12), width=18)
    edit_date.pack(pady=5)

    Label(edit_window, text='Item Bought:', font=('Helvetica', 12), bg=SECONDARY_COLOR).pack(pady=10)
    edit_item_bought = StringVar(value=values[2])
    entry_item_bought = ttk.Entry(edit_window, textvariable=edit_item_bought, width=25)
    entry_item_bought.pack(pady=5)

    Label(edit_window, text='Amount (₹):', font=('Helvetica', 12), bg=SECONDARY_COLOR).pack(pady=10)
    edit_amnt = StringVar(value=str(values[3]))
    entry_amnt = ttk.Entry(edit_window, textvariable=edit_amnt, width=25)
    entry_amnt.pack(pady=5)

    Label(edit_window, text='Mode of Payment:', font=('Helvetica', 12), bg=SECONDARY_COLOR).pack(pady=10)
    edit_MoP = StringVar(value=values[4])
    payment_modes = ['Cash', 'Cheque', 'Credit Card', 'Debit Card', 'Paytm', 'Google Pay', 'Razorpay']
    edit_dd = ttk.Combobox(edit_window, textvariable=edit_MoP, values=payment_modes, state='readonly', width=22)
    edit_dd.pack(pady=5)

    def save_changes():
        new_date = edit_date.get_date()
        new_item_bought = edit_item_bought.get().strip()
        new_amnt_str = edit_amnt.get().strip()
        new_MoP = edit_MoP.get()

        # Debug: Print new values
        print(f"New Values: Date={new_date}, ItemBought={new_item_bought}, Amount={new_amnt_str}, ModeOfPayment={new_MoP}")

        if not new_date or not new_item_bought or not new_amnt_str or not new_MoP:
            mb.showerror('Fields Empty', "Please fill all the fields.")
            return

        try:
            new_amnt = float(new_amnt_str)
            if new_amnt <= 0:
                mb.showerror('Invalid Amount', "Amount must be greater than zero.")
                return
        except ValueError:
            mb.showerror('Invalid Input', "Please enter a valid amount.")
            return

        # Calculate the difference in amount
        old_amount = values[3]
        difference = new_amnt - old_amount
        current_balance = get_current_balance()

        # Debug: Print balance and difference
        print(f"Old Amount: {old_amount}, New Amount: {new_amnt}, Difference: {difference}, Current Balance: {current_balance}")

        if difference > current_balance:
            mb.showerror('Insufficient Balance', "Cannot increase the expense beyond the current balance.")
            return

        try:
            # Update the expense
            cursor.execute(
                '''
                UPDATE ExpenseTracker 
                SET Date = ?, ItemBought = ?, Amount = ?, ModeOfPayment = ?
                WHERE ID = ?
                ''',
                (new_date, new_item_bought, new_amnt, new_MoP, values[0])
            )
            connector.commit()

            # Update the balance
            set_balance(current_balance - difference)

            # Refresh the expense list
            list_all_expenses()

            mb.showinfo('Expense Updated', 'The expense has been updated successfully.')
            edit_window.destroy()

            # Debug: Confirm update
            print(f"Expense ID {values[0]} updated successfully.")

        except sqlite3.Error as e:
            mb.showerror('Database Error', f"An error occurred while updating the expense: {e}")
            print(f"SQLite Error: {e}")

    Button(edit_window, text='Save Changes', command=save_changes, style='TButton').pack(pady=20)

def clear_fields():
    date.set_date(datetime.datetime.now().date())
    item_bought.set('')
    amnt.set(0.0)
    MoP.set('Cash')
    for item in table.selection():
        table.selection_remove(item)

def add_to_buy_item():
    item = to_buy_entry.get().strip()
    if not item or item == "Enter item":
        mb.showerror('Input Error', 'Please enter an item name.')
        return
    try:
        cursor.execute('INSERT INTO ToBuyList (ItemName) VALUES (?)', (item,))
        connector.commit()
        list_to_buy_items()
        to_buy_entry.delete(0, END)
        to_buy_entry.insert(0, "Enter item")
        to_buy_entry.config(fg='grey')
        mb.showinfo('Item Added', f'"{item}" has been added to your To-Buy list.')
        # Debug: Confirm addition
        print(f"Added To-Buy Item: {item}")
    except sqlite3.Error as e:
        mb.showerror('Database Error', f"An error occurred while adding the item: {e}")
        print(f"SQLite Error: {e}")

def remove_to_buy_item():
    selected = to_buy_table.selection()
    if not selected:
        mb.showerror('No Selection', 'Please select an item to remove.')
        return

    selected_item = to_buy_table.item(selected[0])
    item_id = selected_item['values'][0]
    item_name = selected_item['values'][1]

    confirm = mb.askyesno('Confirm Deletion', f"Are you sure you want to remove '{item_name}' from your To-Buy list?")
    if confirm:
        try:
            cursor.execute('DELETE FROM ToBuyList WHERE ID = ?', (item_id,))
            connector.commit()
            list_to_buy_items()
            mb.showinfo('Deleted', f'"{item_name}" has been removed from your To-Buy list.')
            # Debug: Confirm deletion
            print(f"Removed To-Buy Item: ID={item_id}, Name={item_name}")
        except sqlite3.Error as e:
            mb.showerror('Database Error', f"An error occurred while removing the item: {e}")
            print(f"SQLite Error: {e}")

def set_balance_manually():
    # Create a new top-level window
    set_balance_window = Toplevel(root)
    set_balance_window.title("Set Balance")
    set_balance_window.geometry("300x150")
    set_balance_window.configure(bg=SECONDARY_COLOR)
    set_balance_window.resizable(0, 0)

    # Label for Entry
    Label(set_balance_window, text="Enter New Balance (₹):", font=('Helvetica', 12), bg=SECONDARY_COLOR).pack(pady=10)

    # Entry for New Balance
    new_balance_var = StringVar()
    entry_new_balance = ttk.Entry(set_balance_window, textvariable=new_balance_var, width=25)
    entry_new_balance.pack(pady=5)

    def update_balance():
        try:
            new_balance = float(new_balance_var.get())
            if new_balance < 0:
                mb.showerror("Invalid Input", "Balance cannot be negative.")
                return
            set_balance(new_balance)
            mb.showinfo("Success", f"Balance updated to ₹{new_balance:.2f}")
            set_balance_window.destroy()
            # Debug: Confirm balance update
            print(f"Balance updated to: {new_balance}")
        except ValueError:
            mb.showerror("Invalid Input", "Please enter a valid number.")
            print("Invalid input for balance update.")

    # Update Button using ttk.Button
    update_button = ttk.Button(set_balance_window, text="Update", command=update_balance)
    update_button.pack(pady=10)



def clear_all_data():
    surety = mb.askyesno('Are you sure?', 'This will delete all records permanently. Are you sure you want to proceed?')
    if surety:
        try:
            # Delete all records from ExpenseTracker
            cursor.execute('DELETE FROM ExpenseTracker')
            # Reset balance to 0
            set_balance(0.0)
            # Clear Expenses Display
            list_all_expenses()
            # Delete all records from ToBuyList
            cursor.execute('DELETE FROM ToBuyList')
            connector.commit()
            list_to_buy_items()
            mb.showinfo('Data Cleared', 'All data has been successfully cleared from the database.')
            # Debug: Confirm data clearance
            print("All data has been cleared from ExpenseTracker and ToBuyList.")
        except sqlite3.Error as e:
            mb.showerror('Database Error', f"An error occurred while clearing data: {e}")
            print(f"SQLite Error: {e}")
    else:
        mb.showinfo('Action Cancelled', 'No records were deleted.')

def on_closing():
    if mb.askokcancel("Quit", "Do you want to quit?"):
        connector.close()
        root.destroy()

In [7]:
# -------------------- GUI Layout -------------------- #

# Top Label
header_frame = ttk.Frame(root, padding=10)
header_frame.pack(side=TOP, fill=X)
header_label = ttk.Label(header_frame, text='Personal Expense Tracker', style='Header.TLabel')
header_label.pack()

# Main Content Frames
main_frame = ttk.Frame(root, padding=20)
main_frame.pack(fill=BOTH, expand=True)

# Left Frame (Expense Entry & Balance)
left_frame = ttk.Frame(main_frame, padding=10, relief=RAISED)
left_frame.grid(row=0, column=0, padx=10, pady=10, sticky='nsew')

# Current Balance Display
balance_label = ttk.Label(left_frame, text=f"Current Balance: ₹{get_current_balance():.2f}", font=('Helvetica', 14, 'bold'))
balance_label.pack(pady=10)

# Set Balance Button
set_balance_btn = ttk.Button(left_frame, text='Set Balance', command=set_balance_manually)
set_balance_btn.pack(pady=5)

# Clear All Data Button
clear_data_btn = ttk.Button(left_frame, text='Clear All Data', command=clear_all_data, style='TButton')
clear_data_btn.pack(pady=5)

# Expense Entry Frame
expense_entry_frame = ttk.LabelFrame(left_frame, text='Add Expense', padding=10)
expense_entry_frame.pack(pady=10, fill='x')

Label(expense_entry_frame, text='Date:', font=('Helvetica', 12)).grid(row=0, column=0, padx=5, pady=5, sticky='w')
date = DateEntry(expense_entry_frame, date=datetime.datetime.now().date(), font=('Helvetica', 12), width=18)
date.grid(row=0, column=1, padx=5, pady=5)

Label(expense_entry_frame, text='Item Bought:', font=('Helvetica', 12)).grid(row=1, column=0, padx=5, pady=5, sticky='w')
item_bought = StringVar()
entry_item_bought = ttk.Entry(expense_entry_frame, textvariable=item_bought, width=25)
entry_item_bought.grid(row=1, column=1, padx=5, pady=5)

Label(expense_entry_frame, text='Amount (₹):', font=('Helvetica', 12)).grid(row=2, column=0, padx=5, pady=5, sticky='w')
amnt = DoubleVar()
entry_amnt = ttk.Entry(expense_entry_frame, textvariable=amnt, width=25)
entry_amnt.grid(row=2, column=1, padx=5, pady=5)

Label(expense_entry_frame, text='Payment Mode:', font=('Helvetica', 12)).grid(row=3, column=0, padx=5, pady=5, sticky='w')
MoP = StringVar(value='Cash')
payment_modes = ['Cash','Esewa/Khalti','Fonepay']
payment_option = ttk.Combobox(expense_entry_frame, textvariable=MoP, values=payment_modes, state='readonly', width=23)
payment_option.grid(row=3, column=1, padx=5, pady=5)

# Buttons for Expense Entry
button_frame = ttk.Frame(left_frame, padding=10)
button_frame.pack(pady=10, fill='x')

add_expense_btn = ttk.Button(button_frame, text='Add Expense', command=add_another_expense)
add_expense_btn.pack(side=LEFT, padx=5, pady=5)

preview_add_btn = ttk.Button(button_frame, text='Preview & Add', command=add_another_expense)
preview_add_btn.pack(side=LEFT, padx=5, pady=5)

clear_fields_btn = ttk.Button(button_frame, text='Clear Fields', command=clear_fields)
clear_fields_btn.pack(side=LEFT, padx=5, pady=5)

# Middle Frame (To-Buy List)
middle_frame = ttk.Frame(main_frame, padding=10, relief=RAISED)
middle_frame.grid(row=0, column=1, padx=10, pady=10, sticky='nsew')

# To-Buy List Label
to_buy_label = ttk.Label(middle_frame, text='To-Buy List', font=('Helvetica', 16, 'bold'), foreground=PRIMARY_COLOR)
to_buy_label.pack(pady=10)

# Entry for To-Buy Items
to_buy_entry = Entry(middle_frame, font=('Helvetica', 12), width=30, fg='grey')
to_buy_entry.insert(0, "Enter item")
to_buy_entry.pack(pady=5)

def clear_to_buy_entry(event):
    if to_buy_entry.get() == "Enter item":
        to_buy_entry.delete(0, END)
        to_buy_entry.config(fg='black')

to_buy_entry.bind("<FocusIn>", clear_to_buy_entry)

# Buttons for To-Buy List
to_buy_button_frame = ttk.Frame(middle_frame, padding=5)
to_buy_button_frame.pack(pady=5)

add_item_btn = ttk.Button(to_buy_button_frame, text='Add Item', command=add_to_buy_item)
add_item_btn.pack(side=LEFT, padx=5)

remove_item_btn = ttk.Button(to_buy_button_frame, text='Remove Selected', command=remove_to_buy_item)
remove_item_btn.pack(side=LEFT, padx=5)

# To-Buy List Display
to_buy_table = ttk.Treeview(middle_frame, columns=('ID', 'ItemName'), show='headings', height=15)
to_buy_table.heading('ID', text='ID')
to_buy_table.heading('ItemName', text='Item Name')
to_buy_table.column('ID', width=50, anchor=CENTER)
to_buy_table.column('ItemName', width=200, anchor=W)

# Scrollbar for To-Buy List
to_buy_scroll = Scrollbar(middle_frame, orient=VERTICAL, command=to_buy_table.yview)
to_buy_table.configure(yscroll=to_buy_scroll.set)
to_buy_scroll.pack(side=RIGHT, fill=Y)
to_buy_table.pack(pady=10)

list_to_buy_items()

# Right Frame (Expenses Display)
right_frame = ttk.Frame(main_frame, padding=10, relief=RAISED)
right_frame.grid(row=0, column=2, padx=10, pady=10, sticky='nsew')

# Buttons Frame
expenses_button_frame = ttk.Frame(right_frame, padding=5)
expenses_button_frame.pack(pady=5)

edit_expense_btn = ttk.Button(expenses_button_frame, text='Edit Selected', command=edit_expense)
edit_expense_btn.pack(side=LEFT, padx=5)

# Treeview for Expenses
table = ttk.Treeview(right_frame, columns=('ID', 'Date', 'ItemBought', 'Amount', 'ModeOfPayment'), show='headings', selectmode='browse')
table.heading('ID', text='S.No.')
table.heading('Date', text='Date')
table.heading('ItemBought', text='Item Bought')
table.heading('Amount', text='Amount (₹)')
table.heading('ModeOfPayment', text='Payment Mode')

# Define column widths
table.column('ID', width=50, anchor=CENTER)
table.column('Date', width=100, anchor=CENTER)
table.column('ItemBought', width=200, anchor=W)
table.column('Amount', width=100, anchor=E)
table.column('ModeOfPayment', width=120, anchor=W)

# Scrollbars for Expenses
expenses_scroll_y = Scrollbar(right_frame, orient=VERTICAL, command=table.yview)
expenses_scroll_x = Scrollbar(right_frame, orient=HORIZONTAL, command=table.xview)
table.configure(yscroll=expenses_scroll_y.set, xscroll=expenses_scroll_x.set)
expenses_scroll_y.pack(side=RIGHT, fill=Y)
expenses_scroll_x.pack(side=BOTTOM, fill=X)
table.pack(fill=BOTH, expand=True)

# Populate the expenses
list_all_expenses()

In [8]:
# -------------------- Footer -------------------- #

# Footer Frame
footer_frame = ttk.Frame(root, padding=10)
footer_frame.pack(side=BOTTOM, fill=X)

# Footer Label
footer_label = ttk.Label(footer_frame, text='Built by Abashesh Ranabhat', font=('Helvetica', 10), foreground=TEXT_COLOR, background=SECONDARY_COLOR)
footer_label.pack(side=RIGHT)

# -------------------- Window Closing Protocol -------------------- #

root.protocol("WM_DELETE_WINDOW", on_closing)

# -------------------- Finalize GUI -------------------- #

root.mainloop()